<a href="https://colab.research.google.com/github/nandikonda-vishal21/Machine-learning-projects/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [3]:
# first 5 rows of the dataset
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [4]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
17913,29027,-0.422159,0.231118,1.666711,0.451976,-0.203598,0.097244,-0.039666,0.354218,0.062463,-0.267672,0.909700,-0.078854,-2.186030,0.421409,-0.291018,-0.948596,0.715232,-0.080037,1.303620,-0.035462,0.110909,0.435121,-0.056658,0.265867,-0.548204,0.734013,0.117023,0.130972,9.00,0.0
17914,29030,1.177387,-0.215585,0.202972,0.215323,-0.029312,0.601788,-0.297021,0.188082,0.436370,-0.199326,0.515916,1.246514,0.457754,-0.118306,-0.778808,-0.268287,-0.163576,-0.353358,0.553504,-0.035901,-0.055842,0.075903,-0.187120,-0.717798,0.555294,0.731531,-0.022112,-0.010929,25.00,0.0
17915,29030,-0.553746,0.880858,1.644821,-0.132657,0.120940,-0.267411,0.466892,0.222443,-0.639624,-0.174843,1.671591,0.700839,-0.287896,0.417781,0.406584,0.049476,-0.279782,-0.349309,-0.129659,0.070172,-0.133339,-0.348662,0.029947,0.199962,-0.328384,0.071511,0.275487,0.110195,0.89,0.0
17916,29030,-2.844632,3.717960,-7.165428,4.120419,-2.991039,-2.942326,-4.925187,2.204337,-2.663613,-6.888367,4.027447,-8.030026,-1.340344,-9.756963,1.055533,-5.381310,-8.745070,-2.733419,0.983641,0.800709,0.894495,-0.340246,0.012222,-0.059679,-0.104338,-0.295884,1.326228,0.322688,89.99,0.0
17917,29031,1.050204,0.078269,0.484733,1.349623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# dataset informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17918 entries, 0 to 17917
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    17918 non-null  int64  
 1   V1      17918 non-null  float64
 2   V2      17918 non-null  float64
 3   V3      17918 non-null  float64
 4   V4      17918 non-null  float64
 5   V5      17917 non-null  float64
 6   V6      17917 non-null  float64
 7   V7      17917 non-null  float64
 8   V8      17917 non-null  float64
 9   V9      17917 non-null  float64
 10  V10     17917 non-null  float64
 11  V11     17917 non-null  float64
 12  V12     17917 non-null  float64
 13  V13     17917 non-null  float64
 14  V14     17917 non-null  float64
 15  V15     17917 non-null  float64
 16  V16     17917 non-null  float64
 17  V17     17917 non-null  float64
 18  V18     17917 non-null  float64
 19  V19     17917 non-null  float64
 20  V20     17917 non-null  float64
 21  V21     17917 non-null  float64
 22

In [6]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        1
V6        1
V7        1
V8        1
V9        1
V10       1
V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [7]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    17836
1.0       81
Name: Class, dtype: int64

This Dataset is highly unblanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [9]:
# separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [10]:
print(legit.shape)
print(fraud.shape)

(17836, 31)
(81, 31)


In [11]:
# statistical measures of the data
legit.Amount.describe()

count    17836.000000
mean        67.365025
std        188.754429
min          0.000000
25%          5.490000
50%         15.950000
75%         56.232500
max       7712.430000
Name: Amount, dtype: float64

In [12]:
fraud.Amount.describe()

count      81.000000
mean       98.105926
std       267.464067
min         0.000000
25%         1.000000
50%         1.000000
75%        99.990000
max      1809.680000
Name: Amount, dtype: float64

In [13]:
# compare the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,13891.131980,-0.203851,0.228560,0.840631,0.263581,-0.117213,0.112151,-0.110725,-0.008454,0.752065,-0.215358,0.712585,-1.186843,0.781491,0.672759,-0.082770,0.008721,0.354174,-0.026752,-0.073561,0.032646,-0.054930,-0.144997,-0.036871,0.015838,0.118904,0.035813,0.010352,0.006670,67.365025
1.0,16833.074074,-9.315066,6.779465,-13.052870,6.451318,-6.557541,-2.602613,-9.012856,4.730365,-3.087651,-7.499402,6.173253,-9.366936,0.530956,-8.907345,-0.187449,-5.282956,-8.522760,-3.021430,0.395198,0.756424,0.549752,-0.423495,-0.398104,-0.277836,0.355688,0.177616,0.940693,0.090007,98.105926


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions and Fraudulent Transactions

Number of Fraudulent Transactions --> 81

In [14]:
legit_sample = legit.sample(n=81)

Concatenating two DataFrames

In [15]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [16]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
9523,14210,1.304508,0.012626,0.379380,-0.119518,-0.059289,0.202490,-0.476317,0.006686,1.445606,-0.379275,1.315122,-1.833844,2.291989,1.566681,-0.105253,1.066439,-0.428515,0.722562,0.532076,-0.017522,-0.371258,-0.858712,-0.014121,-0.974645,0.140000,0.810990,-0.093266,-0.013241,10.07,0.0
9864,14537,1.138315,-0.240129,1.437573,0.465644,-1.160277,-0.166390,-0.843793,0.084684,1.990782,-0.540387,2.210664,-1.172572,1.859792,1.142375,-1.229226,0.296517,0.445874,0.282964,0.167010,-0.070403,-0.193643,-0.167125,0.073630,0.554375,0.040756,0.887890,-0.062157,0.004909,18.11,0.0
2266,1810,-1.437818,1.596102,0.933307,-0.073344,-0.177388,-0.867138,0.501580,0.422381,-0.418939,0.014686,0.081677,0.487803,0.168018,0.468261,0.855740,-0.253558,0.227146,-1.274632,-0.310218,0.209707,-0.415869,-1.130850,0.230632,0.349254,-0.463737,-0.155975,0.422684,0.253025,10.78,0.0
12489,21875,-0.735799,-0.756565,1.817059,-2.343351,-1.154450,-0.964382,-0.076478,-0.198702,1.602727,-1.524285,0.344857,-2.382604,1.617829,1.376325,1.002277,-3.105290,1.532308,2.086617,1.393516,-0.000706,-0.352723,-0.336850,0.034177,0.308526,0.437966,-0.593838,0.094665,0.095725,119.98,0.0
8059,11006,-1.669155,1.274485,-0.909782,-0.201197,2.940372,3.177675,0.089710,0.630292,1.362083,1.011121,0.782625,-2.750534,1.587114,1.635548,0.444371,-0.227773,0.045166,0.575497,0.613439,0.300013,-0.278845,-0.229555,-0.148068,0.943047,0.087189,-0.393307,0.295081,0.603495,14.15,0.0


In [17]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
17317,28625,-27.848181,15.598193,-28.923756,6.418442,-20.346228,-4.828202,-19.210896,18.329406,-3.668735,-8.009159,4.303096,-6.008660,1.138969,-5.580447,-0.110737,-5.491073,-11.588544,-4.715420,0.734573,1.697856,1.802149,-2.062934,-1.269843,0.165409,1.999499,-0.211059,1.324809,0.388090,99.99,1.0
17366,28658,-28.524268,15.876923,-29.468732,6.447591,-20.786000,-4.865613,-19.501084,18.748872,-3.642990,-7.939642,4.184674,-5.835075,1.215960,-5.330144,-0.118631,-5.367775,-11.431757,-4.696924,0.692688,1.706889,1.805770,-2.119376,-1.317450,0.169846,2.051687,-0.210502,1.301734,0.380246,99.99,1.0
17407,28692,-29.200329,16.155701,-30.013712,6.476731,-21.225810,-4.902997,-19.791248,19.168327,-3.617242,-7.870122,4.066255,-5.661492,1.292950,-5.079846,-0.126523,-5.244472,-11.274973,-4.678437,0.650807,1.715862,1.809371,-2.175815,-1.365104,0.174286,2.103868,-0.209944,1.278681,0.372393,99.99,1.0
17453,28726,-29.876366,16.434525,-30.558697,6.505862,-21.665654,-4.940356,-20.081391,19.587773,-3.591491,-7.800598,3.947840,-5.487911,1.369940,-4.829552,-0.134412,-5.121162,-11.118190,-4.659956,0.608930,1.724779,1.812954,-2.232252,-1.412803,0.178731,2.156042,-0.209385,1.255649,0.364530,99.99,1.0
17480,28755,-30.552380,16.713389,-31.103685,6.534984,-22.105532,-4.977692,-20.371514,20.007208,-3.565738,-7.731071,3.829427,-5.314332,1.446930,-4.579264,-0.142299,-4.997848,-10.961410,-4.641482,0.567056,1.733644,1.816520,-2.288686,-1.460544,0.183179,2.208209,-0.208824,1.232636,0.356660,99.99,1.0


In [18]:
new_dataset['Class'].value_counts()

1.0    81
0.0    81
Name: Class, dtype: int64

In [19]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,14985.111111,0.007009,0.373252,0.853733,0.288314,-0.156894,0.095745,-0.143337,-0.129707,0.895332,-0.249366,0.782098,-1.318081,1.088745,0.811220,-0.197830,0.045458,0.426694,-0.010242,-0.093019,-0.020933,0.034954,-0.171059,-0.012624,-0.039382,0.089227,0.079749,-0.003760,0.035715,49.013951
1.0,16833.074074,-9.315066,6.779465,-13.052870,6.451318,-6.557541,-2.602613,-9.012856,4.730365,-3.087651,-7.499402,6.173253,-9.366936,0.530956,-8.907345,-0.187449,-5.282956,-8.522760,-3.021430,0.395198,0.756424,0.549752,-0.423495,-0.398104,-0.277836,0.355688,0.177616,0.940693,0.090007,98.105926


Splitting the data into Features & Targets

In [20]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [21]:
print(X)

        Time         V1         V2  ...       V27       V28  Amount
9523   14210   1.304508   0.012626  ... -0.093266 -0.013241   10.07
9864   14537   1.138315  -0.240129  ... -0.062157  0.004909   18.11
2266    1810  -1.437818   1.596102  ...  0.422684  0.253025   10.78
12489  21875  -0.735799  -0.756565  ...  0.094665  0.095725  119.98
8059   11006  -1.669155   1.274485  ...  0.295081  0.603495   14.15
...      ...        ...        ...  ...       ...       ...     ...
17317  28625 -27.848181  15.598193  ...  1.324809  0.388090   99.99
17366  28658 -28.524268  15.876923  ...  1.301734  0.380246   99.99
17407  28692 -29.200329  16.155701  ...  1.278681  0.372393   99.99
17453  28726 -29.876366  16.434525  ...  1.255649  0.364530   99.99
17480  28755 -30.552380  16.713389  ...  1.232636  0.356660   99.99

[162 rows x 30 columns]


In [22]:
print(Y)

9523     0.0
9864     0.0
2266     0.0
12489    0.0
8059     0.0
        ... 
17317    1.0
17366    1.0
17407    1.0
17453    1.0
17480    1.0
Name: Class, Length: 162, dtype: float64


Split the data into Training data & Testing Data

In [23]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [24]:
print(X.shape, X_train.shape, X_test.shape)

(162, 30) (129, 30) (33, 30)


Model Training

Logistic Regression

In [25]:
model = LogisticRegression()

In [26]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Model Evaluation

Accuracy Score

In [27]:
# accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [28]:
print('Accuracy on Training data : ', training_data_accuracy)

Accuracy on Training data :  0.9922480620155039


In [29]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [30]:
print('Accuracy score on Test Data : ', test_data_accuracy)

Accuracy score on Test Data :  0.9090909090909091
